#Финальная работа. Часть 1. Сбор данных для дальнейшего анализа и выбора части призноков и снижения используемой оперативной памяти.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Подключение Spark сессии

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()


In [ ]:
spark

Загрузим первый файл из 11 train_data_0.pq

In [ ]:
import pyarrow.parquet as pq
import pyspark.sql.functions as f
parquetFile_0 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_0.pq")

In [ ]:
parquetFile_0.show()

+---+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
| id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_fclose|p

Загрузим все остальные файлы train_data 1-11

In [ ]:
parquetFile_1 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_1.pq")
parquetFile_2 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_2.pq")
parquetFile_3 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_3.pq")
parquetFile_4 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_4.pq")
parquetFile_5 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_5.pq")
parquetFile_6 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_6.pq")
parquetFile_7 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_7.pq")
parquetFile_8 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_8.pq")
parquetFile_9 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_9.pq")
parquetFile_10 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_10.pq")
parquetFile_11 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_11.pq")

Объеденим все в один файл

In [ ]:
df = parquetFile_0.union(parquetFile_1).union(parquetFile_2).union(parquetFile_3).union(parquetFile_4).union(parquetFile_5).union(parquetFile_6)\
.union(parquetFile_7).union(parquetFile_8).union(parquetFile_9).union(parquetFile_10).union(parquetFile_11)
df.show()

+---+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
| id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_fclose|p

In [ ]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- rn: long (nullable = true)
 |-- pre_since_opened: long (nullable = true)
 |-- pre_since_confirmed: long (nullable = true)
 |-- pre_pterm: long (nullable = true)
 |-- pre_fterm: long (nullable = true)
 |-- pre_till_pclose: long (nullable = true)
 |-- pre_till_fclose: long (nullable = true)
 |-- pre_loans_credit_limit: long (nullable = true)
 |-- pre_loans_next_pay_summ: long (nullable = true)
 |-- pre_loans_outstanding: long (nullable = true)
 |-- pre_loans_total_overdue: long (nullable = true)
 |-- pre_loans_max_overdue_sum: long (nullable = true)
 |-- pre_loans_credit_cost_rate: long (nullable = true)
 |-- pre_loans5: long (nullable = true)
 |-- pre_loans530: long (nullable = true)
 |-- pre_loans3060: long (nullable = true)
 |-- pre_loans6090: long (nullable = true)
 |-- pre_loans90: long (nullable = true)
 |-- is_zero_loans5: long (nullable = true)
 |-- is_zero_loans530: long (nullable = true)
 |-- is_zero_loans3060: long (nullable = true)
 |

Сгруппируем данные по id клиентов и расчитаем среднее значение по признакам

In [ ]:
df = df.groupby(['id']).mean()
df.show()

+----+-------+-------+---------------------+------------------------+------------------+------------------+--------------------+--------------------+---------------------------+----------------------------+--------------------------+----------------------------+------------------------------+-------------------------------+---------------+-----------------+------------------+------------------+----------------+-------------------+---------------------+----------------------+----------------------+--------------------+------------------+-------------------+----------------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------

In [ ]:
df.count()

3000000

In [ ]:
df = df.drop('avg(id)')

In [ ]:
df = df.to_pandas_on_spark(index_col='id')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:5725: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [ ]:
df = df.round()

In [ ]:
df.columns = df.columns.str.replace('avg(', '').str.replace(')', '')

In [ ]:
df_new = df.to_spark(index_col='id')

In [ ]:
df_new.show(5)

+------+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
|    id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_fc

В итоге получили 3000000 записей. Загрузим файл с целевым признаком flag (дифолт по кредиту).

In [ ]:
train_target = spark.read.format("csv").option("header", "true").load("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_target.csv")
train_target.show()

+---+----+
| id|flag|
+---+----+
|  0|   0|
|  1|   0|
|  2|   0|
|  3|   0|
|  4|   0|
|  5|   0|
|  6|   0|
|  7|   0|
|  8|   0|
|  9|   0|
| 10|   0|
| 11|   0|
| 12|   0|
| 13|   0|
| 14|   0|
| 15|   0|
| 16|   0|
| 17|   0|
| 18|   0|
| 19|   0|
+---+----+
only showing top 20 rows



In [ ]:
train_target.count()

3000000

Объеденим файл с данными и файл с целевым признаком

In [ ]:
from pyspark.sql.functions import *

In [ ]:
final_train = df_new.join(train_target, 'id', "left")
final_train.show()

+----+----+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+----+
|  id|  rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till

Запишем итоговый файл в формат CSV. Данный фай я буду использовать для выбора признаков по важности использовать для этого библиотеку SHAP в другом ноутбуке.

In [ ]:
final_train.coalesce(1).write.mode('overwrite').csv("/content/drive/MyDrive/Ml_Junior/Финальная работа/Итоговая работа_все файлы/Spark_для_отбора_признаков/data_train.csv", header=True)

|

|

|

|

|

|

|

|

|

|

|

|

|